In [1]:
# 기본 패키지 불러오기 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
plt.style.use("seaborn")
sns.set(font_scale=1)
sns.set_style("whitegrid")

# 그래프를 노트북 안에 그리기 위해 설정
%matplotlib inline

# 맥 OS 폰트 깨짐 방지 코드
from matplotlib import rc
import matplotlib as mpl 
import matplotlib.font_manager as fm 

rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# Inside Airbnb: London Data

## 데이터 불러오기

In [2]:
# 데이터 불러오기
london_lists_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings.csv')
london_lists_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/listings_outline.csv')
london_neighbourhoods_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/neighbourhoods.csv')
london_reviews_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews.csv')
london_reviews_summary_raw = pd.read_csv('../../../../data_weekly/inside_airbnb/London/reviews_outline.csv')

# 분석할 df 별도 복사
london_lists = london_lists_raw.copy()
london_reviews = london_reviews_raw.copy()

# 불러온 df 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 75 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            90852 non-null  int64  
 1   listing_url                                   90852 non-null  object 
 2   scrape_id                                     90852 non-null  int64  
 3   last_scraped                                  90852 non-null  object 
 4   source                                        90852 non-null  object 
 5   name                                          90852 non-null  object 
 6   description                                   87851 non-null  object 
 7   neighborhood_overview                         47521 non-null  object 
 8   picture_url                                   90842 non-null  object 
 9   host_id                                       90852 non-null 

### 데이터 불러온 최종일자 확인 

In [4]:
london_lists_raw['last_scraped'].unique()

array(['2024-03-21', '2024-03-20', '2024-03-22', '2024-03-19'],
      dtype=object)

슈퍼호스트 선정 기준 날짜 년 4회(분기별)
- 1월 1일 / 4월 1일 / 7월 1일 / 10월 1일 

## 분석에서 사용하지 않을 컬럼 삭제

In [3]:
# 1차 선별한 컬럼 리스트 
# 1차 선별한 컬럼 27개 + 검증만 하고 삭제하기로 합의한 컬럼 3개('host_has_profile_pic','host_identity_verified', 'has_availability')
# 총 30개 컬럼 선별 

columns_selected = ['id', 'host_id', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', \
    'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

# 선별한 컬럼만 적용
london_lists = london_lists[columns_selected]

# url도 포함한 df (실제 에어비앤비 사이트 들어가서 확인할 용도)
columns_selected_url = ['id', 'listing_url', 'host_id', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost', 'host_total_listings_count', 'host_has_profile_pic', 'host_identity_verified', 'neighbourhood_cleansed', \
    'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights', 'maximum_nights', 'has_availability', 'number_of_reviews', 'number_of_reviews_l30d', 'review_scores_rating', 'review_scores_accuracy', \
    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

london_url = london_lists_raw[columns_selected_url]

# 변수 정리 
london_lists = london_lists.rename(columns={
    'id': '숙소_id',
    'host_id': '호스트_id',
    'host_response_time': '답변_평균시간',
    'host_response_rate': '문의_응답률',
    'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    'host_total_listings_count': '숙소_수',
    'host_has_profile_pic' : '프로필_사진',
    'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    'minimum_nights': '최소_숙박일',
    'maximum_nights': '최대_숙박일',
    'has_availability' : '예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
})

# url 포함 df 변수 정리 
london_url = london_url.rename(columns={
    'id': '숙소_id',
    'listing_url' : '숙소_url',
    'host_id': '호스트_id',
    'host_response_time': '답변_평균시간',
    'host_response_rate': '문의_응답률',
    'host_acceptance_rate': '예약_수락률',
    'host_is_superhost': '슈퍼호스트',
    'host_total_listings_count': '숙소_수',
    'host_has_profile_pic' : '프로필_사진',
    'host_identity_verified' : '호스트_신원',
    'neighbourhood_cleansed': '숙소_지역',
    'room_type': '숙소_유형',
    'accommodates': '수용_인원수',
    'bathrooms': '욕실수',
    'bedrooms': '침실수',
    'beds': '침대수',
    'amenities': '편의시설',
    'price': '숙소_가격',
    'minimum_nights': '최소_숙박일',
    'maximum_nights': '최대_숙박일',
    'has_availability' : '예약_가능여부',
    'number_of_reviews': '리뷰수',
    'number_of_reviews_l30d': '30일_리뷰수',
    'review_scores_rating': '리뷰점수',
    'review_scores_accuracy': '숙소_정확성_리뷰점수',
    'review_scores_cleanliness': '숙소_청결도_리뷰점수',
    'review_scores_checkin': '숙소_체크인_리뷰점수',
    'review_scores_communication': '숙소_소통_리뷰점수',
    'review_scores_location': '숙소_위치_리뷰점수',
    'review_scores_value': '숙소_가격_리뷰점수',
    'reviews_per_month': '평균_리뷰수'
})


# 컬럼명 확인
london_lists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90852 entries, 0 to 90851
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        90852 non-null  int64  
 1   호스트_id       90852 non-null  int64  
 2   답변_평균시간      57797 non-null  object 
 3   문의_응답률       57797 non-null  object 
 4   예약_수락률       65213 non-null  object 
 5   슈퍼호스트        90463 non-null  object 
 6   숙소_수         90847 non-null  float64
 7   프로필_사진       90847 non-null  object 
 8   호스트_신원       90847 non-null  object 
 9   숙소_지역        90852 non-null  object 
 10  숙소_유형        90852 non-null  object 
 11  수용_인원수       90852 non-null  int64  
 12  욕실수          57894 non-null  float64
 13  침실수          78118 non-null  float64
 14  침대수          57837 non-null  float64
 15  편의시설         90852 non-null  object 
 16  숙소_가격        57885 non-null  object 
 17  최소_숙박일       90852 non-null  int64  
 18  최대_숙박일       90852 non-null  int64  
 19  예약_가

## 결측치 처리에 앞서서 리뷰가 0인 컬럼 제거

In [4]:
# 리뷰수가 0인 컬럼 조건 
condition_review_0 = london_lists['리뷰수'] == 0

# 별도의 DF로 저장
london_lists_review_0 = london_lists[condition_review_0]

리뷰가 0인 데이터의 특징 파악

In [5]:
# 리뷰수가 0인 컬럼의 특징 파악_리뷰데이터 관련

# 개수: 24,983개
print('리뷰가 0개인 런던 내 숙소 개수: ')
print(london_lists_review_0.shape)
print()

# 리뷰가 0개인데 각 리뷰점수가 Null값이 아닌 숙소 개수 

score_type = ['리뷰점수', '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수', '평균_리뷰수']

for type in score_type:
    print(f'{type} :')
    print(f'{type}가 null이 아닌 데이터의 개수 >> {london_lists_review_0[type].notnull().sum()}개')
    print()

리뷰가 0개인 런던 내 숙소 개수: 
(24983, 30)

리뷰점수 :
리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_정확성_리뷰점수 :
숙소_정확성_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_청결도_리뷰점수 :
숙소_청결도_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_체크인_리뷰점수 :
숙소_체크인_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_소통_리뷰점수 :
숙소_소통_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_위치_리뷰점수 :
숙소_위치_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

숙소_가격_리뷰점수 :
숙소_가격_리뷰점수가 null이 아닌 데이터의 개수 >> 0개

평균_리뷰수 :
평균_리뷰수가 null이 아닌 데이터의 개수 >> 0개



In [6]:
# # 리뷰수가 0인 컬럼의 특징 파악_슈퍼호스트 관련

print(london_lists_review_0['슈퍼호스트'].unique())
print()
print('리뷰슈가 0개인 숙소 중 호스트가 슈퍼호스트인지 여부: ')
print(london_lists_review_0['슈퍼호스트'].value_counts())
print()
print('리뷰수가 0개인 숙소 데이터 중 슈퍼호스트가 null값인 데이터 개수:')
print(f'{london_lists_review_0["슈퍼호스트"].isnull().sum()}개')

['f' 't' nan]

리뷰슈가 0개인 숙소 중 호스트가 슈퍼호스트인지 여부: 
f    23754
t     1171
Name: 슈퍼호스트, dtype: int64

리뷰수가 0개인 숙소 데이터 중 슈퍼호스트가 null값인 데이터 개수:
58개


리뷰가 0개가 아닌 데이터만 사용

In [7]:
# 리뷰 개수가 0이 아닌 컬럼 조건 설정
condition_review = london_lists['리뷰수'] != 0

# 리뷰 개수가 0이 아닌 컬럼으로 df 업데이트 
london_lists = london_lists[condition_review]
london_url = london_url[condition_review]

# print(london_lists[:10])
# print()
print(london_lists.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65869 entries, 0 to 90735
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   숙소_id        65869 non-null  int64  
 1   호스트_id       65869 non-null  int64  
 2   답변_평균시간      45111 non-null  object 
 3   문의_응답률       45111 non-null  object 
 4   예약_수락률       51189 non-null  object 
 5   슈퍼호스트        65538 non-null  object 
 6   숙소_수         65869 non-null  float64
 7   프로필_사진       65869 non-null  object 
 8   호스트_신원       65869 non-null  object 
 9   숙소_지역        65869 non-null  object 
 10  숙소_유형        65869 non-null  object 
 11  수용_인원수       65869 non-null  int64  
 12  욕실수          43758 non-null  float64
 13  침실수          58052 non-null  float64
 14  침대수          43703 non-null  float64
 15  편의시설         65869 non-null  object 
 16  숙소_가격        43728 non-null  object 
 17  최소_숙박일       65869 non-null  int64  
 18  최대_숙박일       65869 non-null  int64  
 19  예약_가

In [8]:
# 문의_응답률 / 예약_수략률 >> 타입변경

london_lists['문의_응답률'] = london_lists['문의_응답률'].str.rstrip('%').astype('float')
london_lists['예약_수락률'] = london_lists['예약_수락률'].str.rstrip('%').astype('float')

# 기초 통계 요약
london_lists[['문의_응답률', '예약_수락률']].describe()

,문의_응답률,예약_수락률
count,45111.000000,51189.000000
mean,94.720334,85.460333
std,16.803026,23.021839
min,0.000000,0.000000
25%,100.000000,80.000000
50%,100.000000,97.000000
75%,100.000000,100.000000
max,100.000000,100.000000


결측치때문에 제대로 된 수치가 나오지 않은 것으로 추정

## 결측치 현황

결측치 처리에 집중해야할 컬럼 선택

In [9]:
# 컬럼 전체 결측치 확인 

london_lists.isnull().sum()
# 전체 개수 : 65,869개 중 

숙소_id              0
호스트_id             0
답변_평균시간        20758
문의_응답률         20758
예약_수락률         14680
슈퍼호스트            331
숙소_수               0
프로필_사진             0
호스트_신원             0
숙소_지역              0
숙소_유형              0
수용_인원수             0
욕실수            22111
침실수             7817
침대수            22166
편의시설               0
숙소_가격          22141
최소_숙박일             0
최대_숙박일             0
예약_가능여부         1333
리뷰수                0
30일_리뷰수            0
리뷰점수               0
숙소_정확성_리뷰점수       31
숙소_청결도_리뷰점수       26
숙소_체크인_리뷰점수       61
숙소_소통_리뷰점수        36
숙소_위치_리뷰점수        61
숙소_가격_리뷰점수        63
평균_리뷰수             0
dtype: int64

### 결측치가 천단위 이상인  컬럼

1. '답변_평균시간', '문의_응답률', '예약_수락률'
* 호스트 커뮤니케이션 관련 컬럼 > 고객의 만족도(좋은 숙소?)에 큰 영향을 미치는 요인 중 하나 
2. '욕실수', '침실수', '침대수' 
*  숙소 가격을 결정하는데 큰 요소가 될 수 있는 카테고리 > 크롤링으로 데이터 값 채워넣기(값이 시간에 영향을 받지 않음)
3. '숙소_가격' : 크롤링으로 채워넣기가 힘듬. 시간에 따른 변동성이 큰 값.
* 숙소 가격에 영향을 미칠 요인 추측 : 위치, 숙소 크기, 숙소 형태(ex. 아파트?) 

### 결측치가 천단위 이하인 컬럼

1. 슈퍼호스트 
2. 예약_가능여부 
3. '숙소_정확성_리뷰점수', '숙소_청결도_리뷰점수', '숙소_체크인_리뷰점수', '숙소_소통_리뷰점수', '숙소_위치_리뷰점수', '숙소_가격_리뷰점수'

# 슈퍼호스트

슈퍼호스트 기준
1. 숙박 10건 이상 호스팅 또는 3건의 예약에 걸쳐 총 100박 이상 호스팅
2. 응답률 90% 이상 유지
3. 예약 취소율 1% 미만 유지. 단, 정상참작이 가능한 상황에 따른 예약 취소는 제외
4. 전체 평점 4.8점 이상 유지(후기 작성 기한인 14일이 지나거나, 그전에라도 게스트와 호스트 양측이 모두 후기를 제출하면 후기는 슈퍼호스트 실적에 계산됩니다).

In [46]:
# 슈퍼호스트 null 값 : 331개
print(f'슈퍼호스트 null값 개수: {london_lists["슈퍼호스트"].isnull().sum()}개')

# 슈퍼호스트 null값의 비율: 0.5% >> 삭제해도 무방 
print(f'슈퍼호스트 null값의 비율: {round((london_lists["슈퍼호스트"].isnull().sum() / london_lists.shape[0]) * 100, 2)}%')

슈퍼호스트 null값 개수: 331개
슈퍼호스트 null값의 비율: 0.5%


In [47]:
# 리뷰가 0개인 숙소를 모아놓은 데이터프레임에도 적용 

# 슈퍼호스트 null 값 : 58개
print(f'슈퍼호스트 null값 개수: {london_lists_review_0["슈퍼호스트"].isnull().sum()}개')

# 슈퍼호스트 null값의 비율: 0.5% >> 삭제해도 무방 
print(f'슈퍼호스트 null값의 비율: {round((london_lists_review_0["슈퍼호스트"].isnull().sum() / london_lists_review_0.shape[0]) * 100, 2)}%')

슈퍼호스트 null값 개수: 58개
슈퍼호스트 null값의 비율: 0.23%


In [11]:
london_lists['슈퍼호스트'].value_counts()

f    52458
t    13080
Name: 슈퍼호스트, dtype: int64

In [53]:
# 슈퍼호스트 null값인 데이터와의 비교를 위한 전체 데이터의 기초 통계 요약
london_lists[['문의_응답률', '예약_수락률', '숙소_수', '리뷰수', '리뷰점수', '평균_리뷰수']].describe()

,문의_응답률,예약_수락률,숙소_수,리뷰수,리뷰점수,평균_리뷰수
count,45111.000000,51189.000000,65869.000000,65869.000000,65869.000000,65869.000000
mean,94.720334,85.460333,44.718654,24.566852,4.706596,0.905957
std,16.803026,23.021839,283.832497,48.687816,0.436051,1.195357
min,0.000000,0.000000,1.000000,1.000000,0.000000,0.010000
25%,100.000000,80.000000,1.000000,3.000000,4.590000,0.160000
50%,100.000000,97.000000,3.000000,9.000000,4.840000,0.490000
75%,100.000000,100.000000,11.000000,25.000000,5.000000,1.140000
max,100.000000,100.000000,5658.000000,1672.000000,5.000000,32.260000


In [12]:
# 슈퍼호스트 null값인 데이터 특징 파악

condition_superhost_null = london_lists['슈퍼호스트'].isnull()
london_lists[condition_superhost_null][['문의_응답률', '예약_수락률', '숙소_수', '리뷰수', '리뷰점수', '평균_리뷰수']].describe()

,문의_응답률,예약_수락률,숙소_수,리뷰수,리뷰점수,평균_리뷰수
count,253.000000,270.000000,331.000000,331.000000,331.000000,331.000000
mean,98.747036,94.644444,26.957704,35.525680,4.675921,1.490242
std,6.103541,14.515710,28.076271,50.373229,0.320133,1.390061
min,50.000000,0.000000,2.000000,1.000000,3.400000,0.010000
25%,100.000000,99.000000,5.000000,4.000000,4.500000,0.380000
50%,100.000000,100.000000,14.000000,14.000000,4.730000,1.050000
75%,100.000000,100.000000,36.000000,48.500000,4.970000,2.155000
max,100.000000,100.000000,123.000000,342.000000,5.000000,6.860000


### 슈퍼호스트가 null값인 데이터 특징? 
1. 호스트가 여러 숙소를 운영하고 있다. (숙소수 중위수 14개, 미니멈 2개, q1 5개)
2. 리뷰수가 많다. (중위수 14개)
3. 예약 수략률과 문의 응답률이 모두 좋다 (호스트 단위 집계 항목)

실제 데이터를 좀 봐볼까?

In [13]:
london_url[condition_superhost_null][['숙소_url', '호스트_id', '문의_응답률', '예약_수락률', '숙소_수', '리뷰수', '리뷰점수', '평균_리뷰수']].sort_values(by = ['숙소_수', '리뷰수'], ascending=[False, True])[:30]

,숙소_url,호스트_id,문의_응답률,예약_수락률,숙소_수,리뷰수,리뷰점수,평균_리뷰수
78819,https://www.airbnb.com/rooms/1022410255586290392,60524679,98%,100%,123.0,4,4.25,1.05
30653,https://www.airbnb.com/rooms/34884624,60524679,98%,100%,123.0,6,4.33,0.10
12449,https://www.airbnb.com/rooms/14754559,60524679,98%,100%,123.0,147,4.41,1.61
13539,https://www.airbnb.com/rooms/16328145,60524679,98%,100%,123.0,154,4.38,1.76
13538,https://www.airbnb.com/rooms/16327915,60524679,98%,100%,123.0,158,4.42,1.79
11022,https://www.airbnb.com/rooms/13541121,60524679,98%,100%,123.0,159,4.48,1.70
12216,https://www.airbnb.com/rooms/14483249,60524679,98%,100%,123.0,162,4.43,1.75
11762,https://www.airbnb.com/rooms/14092256,60524679,98%,100%,123.0,164,4.38,1.79
10670,https://www.airbnb.com/rooms/13422389,60524679,98%,100%,123.0,167,4.45,1.77
12448,https://www.airbnb.com/rooms/14754229,60524679,98%,100%,123.0,191,4.45,2.09


엥? 호스트가 같은 값이 많네?

In [51]:
# 슈퍼호스트가 Null값인 데이터에서 호스트가 같은 데이터가 몇개나 있지? 

london_lists[condition_superhost_null]['호스트_id'].value_counts()

494256171    21
239389535    15
268416645    14
154555136    11
4086714      11
             ..
54049469      1
78322712      1
10750879      1
3727368       1
562069730     1
Name: 호스트_id, Length: 130, dtype: int64

In [52]:
# 호스트가 겹치는 숙소(슈퍼호스트가 null값인) 숫자 세기 

# 슈퍼호스트가 Null값인 호스트는 총 몇 명? : 
print(f'슈퍼호스트가 Null값인 호스트의 수 : {london_lists[condition_superhost_null]["호스트_id"].nunique()}명')
print(f'슈퍼호스트가 Null값인 숙소의 수 : {london_lists[condition_superhost_null]["숙소_id"].nunique()}개')
print('-'*60)

# 조건 설정 
condition_result_host_1 = (london_lists[condition_superhost_null]['호스트_id'].value_counts() == 1)

# True와 False 값의 개수를 각각 세기
true_count = condition_result_host_1.sum()
false_count = (~condition_result_host_1).sum()

# 출력
print()
print(f"겹치지 않은 호스트의 수: {true_count}명")
print(f"겹치는 호스트의 수: {false_count}명")

슈퍼호스트가 Null값인 호스트의 수 : 130명
슈퍼호스트가 Null값인 숙소의 수 : 331개
------------------------------------------------------------

겹치지 않은 호스트의 수: 83명
겹치는 호스트의 수: 47명


In [15]:
london_lists[condition_superhost_null]['호스트_id'].value_counts()[:30]

494256171    21
239389535    15
268416645    14
154555136    11
4086714      11
30362264     10
127463633    10
329579576    10
60524679     10
183088797     9
157295299     8
23966399      7
67938882      6
456934029     6
68502358      5
423475816     5
509171821     5
232786181     5
190939015     5
107983837     4
196192499     4
401748044     4
35351106      4
462011865     3
144834468     3
97874184      3
13792962      3
235963197     3
8419096       3
267646946     3
Name: 호스트_id, dtype: int64

숙소 개수는 미니멈이 2개 였으니까 전체 데이터에서 슈퍼호스트가 null값인 호스트의 다른 숙소가 있나 한 번 확인해보자.

In [49]:
# 예시로 직접 찾아보자 

# 리뷰 없는 숙소까지 합쳐서 보자
london = pd.concat([london_lists, london_lists_review_0], axis = 0)
print(london.shape)

# 호스트 아이디별 숙소 찾는 조건 설정
condition_host_id = london['호스트_id'] == 494256171
print(london[condition_host_id].shape)

# # 호스트 아이디별 찾은 숙소에서 슈퍼호스트가 null값이 아닌 숙소 찾기 
# condition_all_superhost_notnull = london['슈퍼호스트'].notnull()
# london[condition_host_id & condition_all_superhost_notnull]

(90852, 30)
(22, 30)


In [42]:
# 슈퍼호스트가 null값인 호스트 ID 개수 
print(f'슈퍼호스트가 null값인 호스트 ID 개수 : {london_lists[condition_superhost_null]["호스트_id"].nunique()}개')
print()

# 슈퍼 호스트가 null값인 호스트 아이디 리스트 생성 
superhost_null_호스트ID_list = london_lists[condition_superhost_null]['호스트_id'].unique().tolist()

# 슈퍼 호스트가 null값인 호스트_ID에 등록된 모든 숙소 가지고 오기 

combined_df_superhost_null = pd.DataFrame()

for host_id in superhost_null_호스트ID_list:
    condition_host_id = london['호스트_id'] == host_id
    filtered_df = london[condition_host_id]
    combined_df_superhost_null = pd.concat([combined_df_superhost_null, filtered_df], axis = 0, ignore_index=True)

print(f'슈퍼호스트가 null값인 호스트 ID가 가지고 있는 숙소의 총 개수: {combined_df_superhost_null["숙소_id"].nunique()}개')
print(f'슈퍼호스트가 null값인 호스트 ID가 가지고 있는 모든 숙소 데이터에서 슈퍼호스트가 null값이 아닌 숙소의 개수 : {combined_df_superhost_null["슈퍼호스트"].notnull().sum()}개')

슈퍼호스트가 null값인 호스트 ID 개수 : 130개

슈퍼호스트가 null값인 호스트 ID가 가지고 있는 숙소의 총 개수: 360개
슈퍼호스트가 null값인 호스트 ID가 가지고 있는 모든 숙소 데이터에서 슈퍼호스트가 null값이 아닌 숙소의 개수 : 0개


* 슈퍼호스트가 null값인 호스트 ID가 가지고 있는 숙소의 총 개수: 360개

<br />

* 슈퍼호스트 null값인 숙소의 개수(review 수가 0이 아닌 df): 331개
* 슈퍼호스트 null값 개수(review 수가 0인 df): 58개

<br />

* 이거 왜 개수가 안맞지?

### 슈퍼호스트 컬럼 / 전처리 방향 설정 결론 

- null 값을 삭제해도 무방하다. (비율이 0.5%)
- 결측치를 일괄 대치하려면 최빈값으로 대치하는 것이 맞을듯
- 결측치를 실제 데이터와 비교해보는 방법도 있다. 
    - NGD 기법 또는 크롤링 